# Lecture 8

In [ ]:
# Please inlcude your names below
# Also, please edit the name of the file and include the names of the two(or three) people answering

# Pair answering the assignment: Jialu Liu, Michael Blum
# Pair giving feedback: Our feedback group is not working on the assignments (yet)

## Selenium Download
For the next exercises you will have to download selenium. 

You can read more about the webdriver here (https://chromedriver.chromium.org), but if you want to go straight to the download, go to https://chromedriver.storage.googleapis.com/index.html?path=80.0.3987.106/ and download your version. 

Moreover, in your terminal type `pip install selenium`. 

Once this is done, you should be able to run:
- `from selenium import webdriver`
- `browser = webdriver.Chrome([the path where you put the googlechromedriver])`

In case of any issues, the https://chromedriver.chromium.org website has some straightforward info on common bugs. 


In [3]:
# from selenium import webdriver

### 1. Rate limiting

1. By now, you are familiar with 3 APIs, namely Google Books, NYT, and Dribble. For each one, find and copy the rules about rate limits. Next, pick one and try to exceed the rate limit; explain what you do and what reaction you get from the API.

Google Books: ~100k/day
    
NYT: 4000 requests per day and 10 per minute

Dribbble: 10000 per day and 60 per minute

2. In the next problem you will check how many requests you can send to Google Search before getting blocked. Websites protect themselves from automated crawling by checking requests that come from the same computer in a small time frame and after a while, they won't respond to the request. A valid response would be "Response 200", which you can see if you just print the response of `requests.get('https://www.google.com/search?q=zurich')`. 

The question is
a) how many requests does it take to get blocked (when you first get a response other than 200)?
b) What is the number of a blocked response and what does it exactly stand for (Google response XXX). If you still can:) 

In [33]:
# I first tried it with the google books API and afterwards with the lichess API. In both cases I couldn't manage
# to break the rate limit. The loop below sends about 1 request per second. I stopped it after 2 hours^^
# eventually I would have gotten a 429 response (Too Many Requests).

import requests
import json
from pprint import pprint
import json

maxRequests = 5
for i in range(maxRequests):
    print('request number', i+1)
    response = requests.get('https://lichess.org/api/user/DrDrunkenstein')
    if response.status_code != 200:
        print(response)
        pprint(json.loads(response.text))
        break

request number 1
request number 2
request number 3
request number 4
request number 5


### 2. Selenium sessions

Go to a website of your choice where you have an account. It can for example be the New York Times APi website where you created a login last time but also tutti.ch, comparis, whatever simple website you often use.

Using Selenium create a session where you 
1. go to the main website 
2. log in 
3. click on an element of your choice 
4. scroll to the bottom of the page
5. then save the page. 

When logging in, you will have to find the name of the login form and submit your credentials to it and then click the login button. Here you find an example for a login using selenium but in case you decide to use this help, Facebook should not be your chosen website. https://crossbrowsertesting.com/blog/test-automation/automate-login-with-selenium/

In [4]:
from selenium import webdriver
import time
driver = webdriver.Chrome()

def lichess_login(username, password):
    driver.get('https://lichess.org/login')
    driver.find_element_by_id('form3-username').send_keys(username)
    driver.find_element_by_id ('form3-password').send_keys(password)
    driver.find_element_by_class_name('submit').click()
    time.sleep(3) # page redirect, so we wait some before srolling down
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)
    with open('page.html', 'w') as f:
        f.write(driver.page_source)
    
    
username = 'your_username'
password = 'your_password'
lichess_login(username, password)

UnicodeEncodeError: 'gbk' codec can't encode character '\u2022' in position 949: illegal multibyte sequence

### 3. Measuring personalization

In this exercise you will have to imitate the study described in class on a website of your interest. You will have to measure differences in the content that you receive back from the website under varying treatments. 

You will have to choose a website and a treatment. Use selenium for this exercise as well. 
- As for websites, you can pick an online store, or traveling site, some news site, Google News.. basically try to pick something that you suspect gives different results for different searchers. 
- Examples for treatments would be location, being logged in with an account, history with the website, being on a phone vs a desktop, etc. 
- You can try to pick multiple searches to make sure you are measuring real phenomenon, not only noise
- You can include a control treatment in case you suspect there's A/B testing or noise in how the pages look
- Finally you have to pick a measure for the differences on the page. In case you receive items on a page, for example URLs or products, you can define an overlap metric. In case the page is more unstructured, come up with an explanation for how you define differences.

As your answer, explain which of the above you chose, how you implemented the experiment, and what difference you found in the pages you collected. 

You can find more infor on how to run multiple browsers at the same time here: https://crossbrowsertesting.com/blog/selenium/run-test-multiple-browsers-parallel-selenium/

In [26]:
#I will measure the personalization of 'your website', and the treament is whether to log in or not.

from selenium import webdriver
import time
#driver = webdriver.Chrome()
url = 'https://www.reddit.com/'

def reddit_no_login():
    driver = webdriver.Chrome()
    driver.get(url)
    #time.sleep(20)
    elem = driver.find_elements_by_class_name('_2oEYZXchPfHwcf9mTMGMg8') #the class name may be temporary

    for el in elem:
        if el.text == 'PROMOTED':
            parent = el.find_element_by_xpath('./../../../..')
            ad = parent.find_element_by_tag_name('h3')
            return ad.text
    
ad = reddit_no_login()
print(ad)

Do you like riddles? Wishlist "The Academy" on Steam!


- Site picked: reddit.com
- treatment: being logged in or not
- measure: ads shown as 'promoted' thread (these threads are 'camouflaged' as normal threads. the only indication of paid advertisement is a small 'promoted' tag)
- test setup: We repeated the test until we found three different ads (we got the same ads multiple times).

Results no login:
Wir läuten den Frühling ein und lassen den PlayStation Store mit fetten Angeboten so richtig aufblühen! Spare jetzt bis zu 60% auf zahlreiche Top-Games wie FIFA 20, Days Gone, Mortal Kombat 11 oder Star Wars Jedi: Fallen Order. So gehen Frühlingsgefühle!
Vends aussi ton assortiment en ligne. Dans ton magasin ou sur internet, garde toujours un œil sur ton inventaire. Améliore ton business avec MyCOMMERCE.ch/fr.
Do you like riddles? Wishlist "The Academy" on Steam!

Results with login:
Experience enjoyable JavaScript development with WebStorm. With smart code completion, safe refactoring, and first-class support for Node.js, Angular and React. Download free trial!
Don’t watch another boring video on secure coding. Try our gamified challenges and put your secure coding skills to the test. Battle the clock and colleagues for bragging rights, if you score more than 5000 points during the trial period, you can flex when we send you a free limited edition t-shirt!
Deine Angebote auch im Online Shop verkaufen. Laden oder Internet – dein Warenlager immer im Blick. Upgrade dein Business mit MyCOMMERCE.ch

#### Conclusion / Observation

- being not logged in resulted in an ad in french. Sample size 
- the theme of the ads were quite similiar. We assume though that in the sample size where we were logged in the ads "got help" from the subreddits the account was linked with (web development in particular).

Congratulations for completing the third notebook! Now it’s time for feedback.
1.	Pass your solution to the other pair in your group.
2.	Include your feedback in the other pair’s notebook. Don’t forget to add your names at the top.
3.	Return the notebook with feedback to the original pairs.
4.	Upload your notebook, with the feedback included by the other pair on OLAT.

You can think of/suggest (among other things)
 - improvements in the code (e.g. readability, efficiency)
 - improvements in the answers (e.g. are they easy to understand, are they correct, how can they be improved?)
 - point out differences (e.g. are there any differences between the responses of the two pairs? if yes what are they, what is the cause, and in which way can they be useful?)
 
Not all suggestions about the type of feedback apply to all types of questions. Try to give feedback in a meaningful and constructive way. 

In [28]:
# Below there is space for giving feedback. This space should be used only by the other pair in your group.

'''
Our feedback partners are not doing the notebooks (yet); therefore we dont have any feedback from them.
'''

'\nOur feedback partners are not doing the notebooks (yet); therefore we dont have any feedback from them.\n'